# Анализ данных Книжного приложения (Выпускной проект: SQL)

## Опишем цели исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

**Цель исследования:**
* Проанализировать данные для формулировки ценностного предложения для книжного приложения.

**План исследования:**
1. Исследуем таблицы;
2. Посчитаем, сколько книг вышло после 1 января 2000 года;
3. Для каждой книги посчитаем количество обзоров и среднюю оценку;
4. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
5. Определим автора с самой высокой средней оценкой книг — учтём только книги с 50 и более оценками;
6. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок;
7. Общий вывод.

## Исследуем таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': '*', # имя пользователя
'pwd': '*', # пароль
'host': '*',
'port': *, # порт подключения
'db': '*'} # название базы данных
connection_string = '*'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = ''' SELECT * 
            FROM books 
            LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = ''' SELECT * 
            FROM authors 
            LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = ''' SELECT * 
            FROM publishers 
            LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [4]:
query = ''' SELECT * 
            FROM ratings 
            LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
query = ''' SELECT * 
            FROM reviews 
            LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
query = ''' SELECT COUNT(review_id) 
            FROM reviews 
            LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,2793


### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Посчитаем, сколько книг вышло после 1 января 2000 года:

In [7]:
query = ''' SELECT COUNT(book_id) AS count_books
            FROM books 
            WHERE publication_date > '01.01.2000' '''
pd.io.sql.read_sql(sql=text(query), con = con)

,count_books
0,819


**Вывод:**
* В нашей базе данных имеется информация о 819 книгах вышедших после 1 января 2000 года;

## Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [17]:
query = ''' SELECT  b.book_id AS book_id,
                    b.title AS book_title,
                    COUNT(DISTINCT(re.review_id)) AS count_reviews,
                    ROUND(AVG(ra.rating),2) AS mean_rating
            FROM books AS b
            LEFT JOIN reviews AS re ON b.book_id = re.book_id
            LEFT JOIN ratings AS ra ON b.book_id = ra.book_id 
            GROUP BY 1
            ORDER BY 3 DESC'''
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,book_title,count_reviews,mean_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


In [14]:
query = ''' WITH one AS (SELECT  b.book_id AS book_id,
                    b.title AS book_title,
                    COUNT(DISTINCT(re.review_id)) AS count_reviews,
                    AVG(ra.rating) AS mean_rating
            FROM books AS b
            LEFT JOIN reviews AS re ON b.book_id = re.book_id
            LEFT JOIN ratings AS ra ON b.book_id = ra.book_id 
            GROUP BY 1
            ORDER BY 3 DESC)
            SELECT SUM(count_reviews)
            FROM one'''
pd.io.sql.read_sql(sql=text(query), con = con)

,sum
0,2793.0


**Вывод:**
* По количеству обзоров "Сумерки" обогнали "Воды слонам", "Стеклянный замок"  и "Гарри Поттера", но средний рейтинг данного произведения оставляет желать лучшего всего ~ 3,66;

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [9]:
query = ''' SELECT p.publisher 
            FROM books b
            INNER JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher
0,Penguin Books


**Вывод:**
* Издательство выпустившее наибольшее число книг толще 50 страниц - `Penguin Books`;

## Определим автора с самой высокой средней оценкой книг — учтём только книги с 50 и более оценками

In [10]:
query = ''' WITH ratings_sorted AS
            (SELECT  book_id,
                    AVG(rating) AS mean_rating
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating_id) >= 50
            ORDER BY 2 DESC)
            
            SELECT  a.author,
                    ROUND(AVG(rs.mean_rating),2) AS rating
            FROM ratings_sorted AS rs
            INNER JOIN books AS b ON rs.book_id=b.book_id
            INNER JOIN authors AS a ON b.author_id=a.author_id
            GROUP BY a.author
            ORDER BY AVG(rs.mean_rating) DESC
            LIMIT 1'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.28


**Вывод:**
* Автор с самой высокой средней оценкой книг (только с учетом книг с 50 и более оценками) - `J.K. Rowling/Mary GrandPré`, средний рейтинг книг этого автора 4,28. "Гарри Поттер" получает много оценок при том достаточно хороших;

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = ''' WITH ratings_sorted AS
            (SELECT username,
                    COUNT(rating_id) AS count_rating
            FROM ratings 
            GROUP BY 1
            HAVING COUNT(rating_id) > 48),
            review_sorted AS 
            (SELECT  username,
                    COUNT(review_id) AS count_reviews
            FROM reviews
            GROUP BY 1)
            SELECT AVG(count_reviews) AS mean_reviews
            FROM ratings_sorted AS ras
            INNER JOIN review_sorted AS res ON ras.username = res.username'''
pd.io.sql.read_sql(sql=text(query), con = con)

,mean_reviews
0,24.0


**Вывод:**
* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

## Общий вывод

Мы проанализировали данные сервиса для чтенитя книг по подписке, такие результаты мы получили:

* В нашей базе данных имеется информация о 819 книгах вышедших после 1 января 2000 года;
* По количеству обзоров "Сумерки" обогнали "Воды слонам", "Стеклянный замок"  и "Гарри Поттера", но средний рейтинг данного произведения оставляет желать лучшего всего ~ 3,66;
* Издательство выпустившее наибольшее число книг толще 50 страниц - `Penguin Books`;
* Автор с самой высокой средней оценкой книг (только с учетом книг с 50 и более оценками) - `J.K. Rowling/Mary GrandPré`, средний рейтинг книг этого автора 4,28. "Гарри Поттер" получает много оценок при том достаточно хороших;
* Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.